## PreProp

In [1]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #GPU 1
import cv2
import json  
import numpy as np
import pandas as pd
import tensorflow as tf
import SimpleITK as sitk
import matplotlib.pyplot as plt

from tqdm import tqdm
from skimage import measure
from scipy import ndimage as ndi
from skimage.filters import roberts
from skimage.measure import label,regionprops
from skimage.segmentation import clear_border
from skimage.morphology import  convex_hull_image
from skimage.morphology import disk, binary_closing

2025-10-31 11:29:18.357735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 11:29:18.442879: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-31 11:29:20.512514: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
#paths dirs

BASE_DIR = os.getcwd()  #'/home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline'
DATA_DIR = os.path.join(BASE_DIR, 'data')  #'/home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/data'
assert os.path.isdir(DATA_DIR) , "❎ data file not exist."
SELECTED_CSV = os.path.join(BASE_DIR, 'csv/selected.csv')  #'/home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/csv/selected.csv'
assert os.path.exists(SELECTED_CSV) , "❎ csv file not exist." 
TEMP_DIR = os.path.join(BASE_DIR, '_temp')  #'/home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/_temp'
os.makedirs(TEMP_DIR, exist_ok=True)
assert os.path.isdir(TEMP_DIR) , "❎ temp file not exist."  

In [3]:
#env 
SUBSET = 0
PATCH_SIZE = 176 # Tile size
N = 3  # 9 patches

#debug
debug = 0
plot = 0
old = 0

In [4]:
#temp dirs

PATCHES_CT_DIR = os.path.join(TEMP_DIR, "patches_ct")
os.makedirs(PATCHES_CT_DIR, exist_ok=True)
PATCHES_POS_DIR = os.path.join(TEMP_DIR, "patches_pos")
os.makedirs(PATCHES_POS_DIR, exist_ok=True)

In [5]:
#load_dicom_series
def load_dicom_series(series_dir):
    """
    用 SimpleITK 讀取單一病例資料夾（3mm/{case}）下的一整個 DICOM series
    回傳：ct (Z, Y, X)、origin(x,y,z)、spacing(x,y,z)
    """
    series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(series_dir)
    if not series_IDs:
        raise RuntimeError(f"No DICOM series found in: {series_dir}")
    # 預設拿第一個 series（若同一資料夾有多個 series，可自行挑選）
    series_files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(series_dir, series_IDs[0])
    reader = sitk.ImageSeriesReader()
    reader.SetFileNames(series_files)
    image = reader.Execute()
    ct = sitk.GetArrayFromImage(image)              # (Z, Y, X)
    origin = np.array(list(image.GetOrigin()))      # (X, Y, Z)
    spacing = np.array(list(image.GetSpacing()))    # (X, Y, Z)
    return ct, origin, spacing

In [6]:
#hu,wl/ww : normalize
def sanitize_hu(vol_np, pixel_padding_value=None):
    v = vol_np.astype(np.float32, copy=True)
    # 若知道 PPV（很多機器是 -2000 / -3024），一起處理
    if pixel_padding_value is not None and pixel_padding_value <= -1500:
        v[vol_np <= pixel_padding_value + 1e-6] = -1024.0
    # 合理裁切（保底處理 -2000 這種哨兵值）
    v[v < -1024.0] = -1024.0
    v[v > 3071.0]  = 3071.0
    return v
def window_to_uint8(v_hu, wl=-600, ww=1500):
    low, high = wl - ww/2, wl + ww/2
    v = np.clip(v_hu, low, high)
    return ((v - low) / ww * 255).astype('uint8')

In [7]:
#crop/padding tile
def linspace_starts(L, tile, n):
    """在長度 L 上均勻取 n 個起點，含 0 與 L-tile。
       例如 L=512,tile=176,n=3 -> [0,168,336]（彼此有 8px 微重疊）"""
    if L <= tile:
        return [0]
    return [int(round(i * (L - tile) / (n - 1))) for i in range(n)]

def safe_crop(arr, y0, x0, size):
    """從 (y0,x0) 切 size×size；若越界則零填充，不做 resize。"""
    H, W = arr.shape
    y1, x1 = y0 + size, x0 + size
    y0c, x0c = max(0, y0), max(0, x0)
    y1c, x1c = min(H, y1), min(W, x1)
    crop = arr[y0c:y1c, x0c:x1c]
    pad_top   = y0c - y0
    pad_left  = x0c - x0
    pad_bot   = y1 - y1c
    pad_right = x1 - x1c
    if pad_top or pad_left or pad_bot or pad_right:
        crop = np.pad(crop, ((pad_top, pad_bot), (pad_left, pad_right)),
                      mode='constant', constant_values=0)
    return crop


In [8]:
#main code
#----get df, case_dir----
df = pd.read_csv(SELECTED_CSV)
df["case_dir"] = df["uid"].map(lambda cid: os.path.join(DATA_DIR, str(cid)))
df = df[df["case_dir"].map(os.path.isdir)].copy()
df["filename"] = df["case_dir"]
case_dirs = np.unique(df["filename"].values)
#------------------------
df

,uid,case_dir,filename


In [9]:
#lung segment cache
def ensure_slice_assets(z_idx: int):
    """確保某個 z 的 (img_norm, extracted_lungs) 已計算並快取。"""
    if z_idx in case_slice_img:
        return

    ct_org = ct_norm[z_idx]
    #單張測試
    im = ct_org.copy() #**
    plot = 0  

    if plot:
        plt.hist(im.flatten(), bins=80, color='c')
        plt.xlabel("Hounsfield Units (HU)")
        plt.ylabel("Frequency")
        plt.show()    

    #print(f'ct2:{ct2.shape},im : {im.shape}')#ct2:(49, 273, 1536),im : (512, 512)
    if plot:
        plt.figure(figsize=(28,7))
        plt.subplot(1,4,1)
        plt.imshow(im, cmap=plt.cm.bone)
    # Step 1: Convert into a binary image.
    binary_thr = im <  175  #80 #on scale 0~255:ct_norm  #-604:on scale HU:ct
    if plot:
        plt.subplot(1,4,2)
        plt.imshow(binary_thr, cmap=plt.cm.bone)
    # Step 2: Remove the blobs connected to the border of the image.
    cleared = clear_border(binary_thr)
    # Step 3: Label the image.
    label_image = label(cleared)
    # Step 4: Keep the labels with 2 largest areas and segment two lungs.
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    labels = []
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:
                    label_image[coordinates[0], coordinates[1]] = 0
            else:
                coordinates = region.coords[0]
                labels.append(label_image[coordinates[0], coordinates[1]])
    else:
        labels = [1, 2]
    # Step 5: Fill in the small holes inside the mask of lungs which we seperate right and left lung. r and l are symbolic and they can be actually left and right!
    r = label_image == labels[0]
    l = label_image == labels[1]
    r_edges = roberts(r)
    l_edges = roberts(l)
    r = ndi.binary_fill_holes(r_edges)
    l = ndi.binary_fill_holes(l_edges)
    # Step 6: convex hull of each lung
    r = convex_hull_image(r)
    l = convex_hull_image(l)
    # Step 7: joint two separated right and left lungs.
    sum_of_lr = r + l
    binary = sum_of_lr > 0
    # Step 8: Closure operation with a disk of radius 10. This operation is
    selem = disk(10)
    binary_c = binary_closing(binary, selem)
    if plot:
        plt.subplot(1,4,3)
        plt.imshow(binary_c, cmap=plt.cm.bone)
    # Step 9: Superimpose the binary mask on the input image.
    get_high_vals = binary_c == 0
    im[get_high_vals] = 0
    if plot:
        plt.subplot(1,4,4)
        plt.imshow(im, cmap=plt.cm.bone)
        plt.savefig("./demo_prepro.png")
        plt.show()
    
    case_slice_img[z_idx] = ct_org.copy()
    case_lungs[z_idx] = im.copy() #extracted_lungs   

In [10]:

# =========================
# 主流程（改為先聚合、後存檔；I/O 與處理管線一致）
# =========================
for case_i, case_dir in tqdm(enumerate(case_dirs), total=len(case_dirs)):
    filename = case_dir.split("/")[-1]
    if plot : print(f"\n➡️ Processing case {case_i}: {filename}")
    pos_file_dir = os.path.join(PATCHES_POS_DIR, filename)
    os.makedirs(pos_file_dir, exist_ok=True)
    assert os.path.isdir(pos_file_dir) , "❎ ct position output dir not exist."  

    out_dir = os.path.join(PATCHES_CT_DIR, filename)
    os.makedirs(out_dir, exist_ok=True)
    assert os.path.isdir(out_dir) , "❎ ct patches output dir not exist."  

    ann_case = df[df["filename"] == case_dir]

    # 讀 DICOM series -> 3D 影像
    print('➡️  Loading DICOM series ...')
    ct, origin, spacing = load_dicom_series(case_dir)     # ct: (Z, Y, X)
    num_z, H, W = ct.shape
    if plot : print(f'num_z  {num_z}')
    ct = ct[::-1].copy()  # 翻轉 Z 軸，讓 z=0 在人體底部（腳底）

    # 與參考程式一致：把整個 volume 正規化到 0~255（之後每張再抽切片）
    print('➡️  Normalizing volume ...')
    ct = sanitize_hu(ct)
    ct_norm = window_to_uint8(ct, wl=-600, ww=1500)

    # ---- 以 z_idx 為 key 的彙整容器（同 slice 多結節做 union）----
    case_lungs = {}          # {z_idx: uint8 extracted_lungs (0~255) shape (512,512)}   # 切割好的肺部 (套上mask的肺)
    case_slice_img = {}      # {z_idx: uint8 resized slice (512,512)}                    # Original 512 dicom slice

    # ---- 逐 annotation 聚合（僅累積，不存檔）----
    for idx in range(0,num_z):
        ensure_slice_assets(idx)

    # ---- 此 case 收尾：一次性存檔（每個 z 只會存一張）----

    for z in sorted(case_lungs.keys()):
        lungs_out = case_lungs[z].astype(np.uint8)  # ensure 已產生

        if not debug:
            # === 切 3x3 共 9 片 176×176 並存 .npy（無任何過濾/特殊處理）===
            H, W = lungs_out.shape
            ys = linspace_starts(H, PATCH_SIZE, N)  # 512 -> [0,168,336]
            xs = linspace_starts(W, PATCH_SIZE, N)

            for ri, y0 in enumerate(ys):
                for ci, x0 in enumerate(xs):
                    tile = safe_crop(lungs_out, y0, x0, PATCH_SIZE)
                    # 正常 512×512 會剛好 9 片、每片 176×176；若不是 512，也會自動邊緣零填充
                    base = f"lungs_{SUBSET}_{case_i}_{z:04d}_r{ri}_c{ci}"
                    np.save(os.path.join(out_dir, base + ".npy"), tile.astype(np.uint8))
                    
                    y1 = min(y0 + PATCH_SIZE, H)  # clamp to slice size in case of padding
                    x1 = min(x0 + PATCH_SIZE, W)
                    info = {
                        "subset":        SUBSET,
                        "case_index":    int(case_i),
                        "case_dir":      str(case_dir),
                        "z_index":       int(z),
                        "r":             int(ri),
                        "c":             int(ci),
                        "tile_size":     int(PATCH_SIZE),
                        "slice_size":    {"H": int(H), "W": int(W)},
                        "slice_coords":  {"y0": int(y0), "x0": int(x0), "y1": int(y1), "x1": int(x1)},
                        "padded":        bool((y1 - y0 < PATCH_SIZE) or (x1 - x0 < PATCH_SIZE)),  # true if safe_crop padded
                        "npy_name":      base + ".npy"
                    }
                    with open(os.path.join(pos_file_dir, base + ".json"), "w") as f:
                        json.dump(info, f, indent=2)

print("✅ Done.")


0it [00:00, ?it/s]

✅ Done.


## MODEL

In [11]:
import numpy as np
import os, glob, json, cv2

from collections import defaultdict
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate , BatchNormalization, ReLU, Add,  SpatialDropout2D, Activation

In [12]:
#Paths

BASE_DIR = os.getcwd()
MODEL_PATH = os.path.join(BASE_DIR, 'model/Resunet_model.h5')
assert os.path.exists(MODEL_PATH) , "❎ model h5 file not exist."  

#Dirs

PATCHES_CT_DIR = os.path.join(TEMP_DIR, "patches_ct")
assert os.path.isdir(PATCHES_CT_DIR) , "❎ patches_ct dir not exist."  
PATCHE_POS_DIR = os.path.join(TEMP_DIR, "patches_pos")
assert os.path.isdir(PATCHE_POS_DIR) , "❎ patches_pos dir not exist."  

In [13]:
# Model Structure
def res_block(x, filters, wd=1e-4, drop=0.0, dilation_rate=(1, 1)):
    """Residual block with optional dilation rate: 3x3 -> BN+ReLU -> 3x3 -> BN + shortcut + ReLU (+ optional SpatialDropout2D)"""
    shortcut = x
    in_ch = K.int_shape(x)[-1]

    # 3x3 dilated conv -> BN -> ReLU
    x = Conv2D(filters, 3, padding="same", dilation_rate=dilation_rate, use_bias=False,
               kernel_initializer="he_normal", kernel_regularizer=l2(wd))(x)
    x = BatchNormalization()(x); x = ReLU()(x)

    # 3x3 conv -> BN
    x = Conv2D(filters, 3, padding="same", dilation_rate=dilation_rate, use_bias=False,
               kernel_initializer="he_normal", kernel_regularizer=l2(wd))(x)
    x = BatchNormalization()(x)

    # 若通道數不同，用 1x1 conv 對齊 shortcut
    if in_ch != filters:
        shortcut = Conv2D(filters, 1, padding="same", use_bias=False,
                          kernel_initializer="he_normal", kernel_regularizer=l2(wd))(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 殘差相加 -> ReLU
    x = Add()([x, shortcut])
    x = ReLU()(x)

    if drop > 0:
        x = SpatialDropout2D(drop)(x)
    return x

def resunet(input_size=(176, 176, 1), wd=1e-4, drop=0.2):
    inputs = Input(input_size)

    # Encoder with increased filters and more residual blocks
    c1 = res_block(inputs,  32, wd, drop=0.0)
    p1 = MaxPooling2D(2)(c1)

    c2 = res_block(p1,     64, wd, drop=0.0)
    p2 = MaxPooling2D(2)(c2)

    c3 = res_block(p2,    128, wd, drop=0.0)
    p3 = MaxPooling2D(2)(c3)

    c4 = res_block(p3,    256, wd, drop=drop)
    p4 = MaxPooling2D(2)(c4)

    # Bottleneck with dilated convolution
    c5 = res_block(p4,    512, wd, drop=drop, dilation_rate=(2, 2))  # Increased feature maps and dilation

    # Decoder with more filters
    u6 = concatenate([Conv2DTranspose(256, 2, strides=2, padding="same")(c5), c4])
    c6 = res_block(u6,    256, wd, drop=drop)

    u7 = concatenate([Conv2DTranspose(128, 2, strides=2, padding="same")(c6), c3])
    c7 = res_block(u7,    128, wd, drop=drop)

    u8 = concatenate([Conv2DTranspose(64, 2, strides=2, padding="same")(c7), c2])
    c8 = res_block(u8,     64, wd, drop=drop)

    u9 = concatenate([Conv2DTranspose(32, 2, strides=2, padding="same")(c8), c1])
    c9 = res_block(u9,     32, wd, drop=0.0)

    # Output layer
    outputs = Conv2D(1, 1, activation="sigmoid", kernel_initializer="he_normal")(c9)
    model = Model(inputs, outputs, name="ResUNet")

    # Compile model with Adam optimizer and Focal Tversky Loss
    opt = Adam(learning_rate=1e-4)
    model.compile(optimizer=opt,
                  loss=focal_tversky_loss,               # or focal_tversky_loss
                  metrics=[focal_tversky_loss])
    return model



In [14]:
# Loss
def tversky(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1.0):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    tp = K.sum(y_true * y_pred)
    fp = K.sum((1-y_true) * y_pred)
    fn = K.sum(y_true * (1-y_pred))
    return (tp + smooth) / (tp + alpha*fp + beta*fn + smooth)

def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    return K.pow(1.0 - tversky(y_true, y_pred), gamma)

In [15]:
%%time
#MODEL
model2 = resunet(input_size=(176,176,1))
model2.load_weights(MODEL_PATH)
model2.compile(optimizer=Adam(learning_rate=2e-4),
                loss=focal_tversky_loss,
                metrics = [tversky,focal_tversky_loss, 'binary_accuracy'])

I0000 00:00:1761881363.861951   37024 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22487 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:40:00.0, compute capability: 7.5


CPU times: user 2.82 s, sys: 1.11 s, total: 3.94 s
Wall time: 3.81 s


In [16]:
%%time
from collections import defaultdict
import os, glob, json, numpy as np, cv2

# ---------- 參數 ----------
BATCH = 128         # 依 VRAM 調（64/128/256）；越大越快到一個甜蜜點
THR = 0.96          # 你的閾值，維持原樣
USE_MMAP = True     # 以避免不必要拷貝

# ---------- 1) 建立 base -> info 的快取（不要逐檔 open json） ----------
pos_infos = {}
for jpath in glob.glob(os.path.join(PATCHE_POS_DIR, "**", "*.json"), recursive=True):
    base = os.path.splitext(os.path.basename(jpath))[0]
    with open(jpath, "r") as jf:
        pos_infos[base] = json.load(jf)

CPU times: user 1.65 s, sys: 691 ms, total: 2.34 s
Wall time: 2.35 s


In [17]:
print(f"[index] loaded {len(pos_infos)} position jsons")
tile_files = sorted(glob.glob(os.path.join(PATCHES_CT_DIR, "**", "lungs_*.npy"), recursive=True))


[index] loaded 49059 position jsons


In [18]:


# ---------- 2) 走訪所有 tiles，但改為批次送進 GPU ----------
tile_files = sorted(glob.glob(os.path.join(PATCHES_CT_DIR, "**", "lungs_*.npy"), recursive=True))

slice_nodules = defaultdict(list)  # key=(case_key, z) -> list[[z, x1, y1, w, h, conf]]

batch_imgs = []         # list[np.ndarray (176,176)]
batch_meta = []         # list[(case_key, z, y0, x0, H, W)]

def flush_batch():
    if not batch_imgs:
        return
    X = np.stack(batch_imgs, axis=0)[..., None].astype(np.float32)   # (B,176,176,1)
    # 建議用 model(...) 取代 predict(...)，減少 Keras 封裝開銷
    probs = model2(X, training=False).numpy()[..., 0]                # (B,176,176)

    for prob, (case_key, z, y0, x0, H, W) in zip(probs, batch_meta):
        m = (prob >= THR).astype(np.uint8)
        if m.max() == 0:
            continue
        num, labels, stats, cents = cv2.connectedComponentsWithStats(m, connectivity=8)
        for k in range(1, num):
            x1 = int(stats[k, cv2.CC_STAT_LEFT]);  y1 = int(stats[k, cv2.CC_STAT_TOP])
            w  = int(stats[k, cv2.CC_STAT_WIDTH]); h  = int(stats[k, cv2.CC_STAT_HEIGHT])
            # map tile -> slice
            xg1 = int(x0 + x1);    yg1 = int(y0 + y1)
            xg2 = int(min(xg1 + w, W)); yg2 = int(min(yg1 + h, H))
            conf = float(prob[labels == k].mean())
            slice_nodules[(case_key, z)].append([int(z), xg1, yg1, int(xg2 - xg1), int(yg2 - yg1), conf])

    batch_imgs.clear()
    batch_meta.clear()

# 批次讀檔 + 推論
for f in tile_files:
    base = os.path.splitext(os.path.basename(f))[0]
    info = pos_infos.get(base)

    if info is None:
        continue
    z  = int(info["z_index"])
    y0 = int(info["slice_coords"]["y0"]); x0 = int(info["slice_coords"]["x0"])
    H  = int(info["slice_size"]["H"]);    W  = int(info["slice_size"]["W"])
    
    
    
    case_key = os.path.basename(os.path.dirname(f))

    # 讀檔並正規化；空白塊可直接略過（省推論）
    if USE_MMAP:
        tile = np.load(f, mmap_mode='r')
        if tile.max() == 0:   # 全黑（大多是肺外區域）
            continue
        tile = tile.astype(np.float32) / 255.0
    else:
        tile = np.load(f).astype(np.float32) / 255.0
        if tile.max() == 0:
            continue

    batch_imgs.append(tile)
    batch_meta.append((case_key, z, y0, x0, H, W))

    if len(batch_imgs) >= BATCH:
        flush_batch()

# 收尾
flush_batch()
print("✅ Done (batched)")


2025-10-31 11:29:30.318463: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301
2025-10-31 11:29:36.108253: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-10-31 11:29:36.193653: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


✅ Done (batched)


In [19]:
%%time
import numpy as np
import pandas as pd

# 顯示設定
DEBUG_PRINT = True        # True：印出每個 slice 的 box；False：只印總結
MAX_SHOW_PER_SLICE = 5    # 每個 slice 最多展示幾個 box（避免刷屏）

# 統計器
total_boxes = 0
slices_with_det = 0
cases = set()
rows = []   # 收集成表格（可選）

# 直接走 items，避免二次索引
for (case_key, z), boxes in sorted(slice_nodules.items(), key=lambda t: (t[0][0], t[0][1])):
    cases.add(case_key)
    n = len(boxes)
    if n == 0:
        # 理論上不會進來（因為只有有偵測才會存 key），但保險處理
        if DEBUG_PRINT:
            print(f"{case_key} | z={z} -> 0 nodules")
        continue

    slices_with_det += 1
    total_boxes += n

    if DEBUG_PRINT:
        print(f"{case_key} | z={z} -> {n} nodules")
        # 轉成 np.array（若每筆都是 6 維，形狀會是 (n, 6)）
        try:
            arr = np.asarray(boxes, dtype=np.float32)
        except Exception:
            # 若形狀不齊，退回逐列印
            arr = None
        if arr is not None and arr.ndim == 2 and arr.shape[1] == 6:
            # 只顯示前幾筆，避免輸出過多
            show_n = min(n, MAX_SHOW_PER_SLICE)
            print(arr[:show_n])
            if n > show_n:
                print(f"... ({n - show_n} more)")
        else:
            # 後備路徑：逐列印
            show_n = min(n, MAX_SHOW_PER_SLICE)
            for i, b in enumerate(boxes[:show_n]):
                print(b)
            if n > show_n:
                print(f"... ({n - show_n} more)")

    # 蒐集成表格列
    for b in boxes:
        # 每筆 b: [z, x1, y1, w, h, conf]
        z_i, x1, y1, w, h, conf = b
        rows.append([case_key, int(z_i), int(x1), int(y1), int(w), int(h), float(conf)])

print(f"✅ {slices_with_det} slices with detections across {len(cases)} cases")
print(f"✅ {total_boxes} nodules")

#SAVE
df_det = pd.DataFrame(rows, columns=["case_key", "z", "x1", "y1", "w", "h", "conf"])
# df_det.to_csv("./target/nodule_detections.csv", index=False)


11161702 | z=56 -> 1 nodules
[[ 56.        249.        247.          6.          4.          0.9920705]]
11161702 | z=65 -> 1 nodules
[[ 65.        123.        295.          1.          1.          0.9839078]]
11161702 | z=66 -> 1 nodules
[[ 66.         109.         253.           9.           9.
    0.99936116]]
11161702 | z=68 -> 1 nodules
[[ 68.       113.       263.         6.         6.         0.992288]]
11161702 | z=69 -> 1 nodules
[[ 69.       115.       264.         6.         7.         0.997506]]
11161702 | z=74 -> 1 nodules
[[ 74.        120.        282.          9.         12.          0.9984463]]
11161702 | z=76 -> 1 nodules
[[ 76.      130.      278.       19.       20.        0.99913]]
11161702 | z=77 -> 1 nodules
[[ 77.       131.       274.        26.        24.         0.999729]]
11161702 | z=78 -> 1 nodules
[[ 78.        130.        274.         27.         25.          0.9997589]]
11161702 | z=79 -> 1 nodules
[[ 79.        131.        275.         26.         24.  

In [20]:
display(df_det.head())


,case_key,z,x1,y1,w,h,conf
0,11161702,56,249,247,6,4,0.992070
1,11161702,65,123,295,1,1,0.983908
2,11161702,66,109,253,9,9,0.999361
3,11161702,68,113,263,6,6,0.992288
4,11161702,69,115,264,6,7,0.997506


## 後處理

In [21]:
import cv2
import numpy as np
import os, json, glob
import matplotlib.pyplot as plt
from collections import defaultdict

In [22]:
#variable

WL, WW = -600, 1500
FLIP_Z = True  # 建 tile 時有翻就設 True，沒有就 False
# ========= 可調參數 =========
IOU_THR_SLICE = 0.3      # 同片 NMS 的 IoU 門檻
CENTER_THR_PX = 12       # 串連時允許的中心距離(像素)
MAX_Z_GAP = 1            # 串連允許跨越的最大 z 間隔（1=相鄰片）
MIN_SLICE_LEN = 2        # 一顆結節至少出現幾片（過低容易是假陽性）
MIN_AREA_PX = 5 * 5      # 最小像素面積（過濾雜點）；可改成用毫米^2
KEEP_TOPK_PER_SLICE = None  # 例如 5；None 表示不限制

#dirs
OTHERS_PATH = os.path.join(BASE_DIR, 'others')  #'/home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/others'
os.makedirs(OTHERS_PATH, exist_ok=True)
assert os.path.isdir(OTHERS_PATH) , "❎ others file not exist."  

In [23]:
# for plots purpose

# (1) 建立 case_key -> case_dir 的映射（用你 position JSON 內的 case_dir）
case2dir = {}
for jpath in glob.glob(os.path.join(PATCHE_POS_DIR, "**", "*.json"), recursive=True):
    with open(jpath, "r") as jf:
        info = json.load(jf)
    case_key = os.path.basename(os.path.dirname(jpath))
    case2dir.setdefault(case_key, info["case_dir"])

# (2) 為了快，不重複讀 DICOM：cache 每個 case 的 volume（ct_norm）
_volume_cache = {}

def get_ct_norm(case_key):
    if case_key in _volume_cache:
        return _volume_cache[case_key]
    case_dir = case2dir[case_key]
    ct, origin, spacing = load_dicom_series(case_dir)  # 你現成的 loader
    if FLIP_Z:
        ct = ct[::-1].copy()
    ct = sanitize_hu(ct)
    ct_norm = window_to_uint8(ct, wl=WL, ww=WW)  # (Z,H,W) uint8
    _volume_cache[case_key] = ct_norm
    return ct_norm

def overlay_one_slice(case_key, z, out_path=None, show=True, alpha=0.30, thickness=2):
    """
    在單一 slice 上畫偵測框，並顯示/存檔。
    slice_nodules[(case_key, z)] 的每筆是 [z, x1, y1, w, h, conf]
    """
    ct_norm = get_ct_norm(case_key)
    assert 0 <= z < ct_norm.shape[0], f"z 超出範圍: {z}/{ct_norm.shape[0]}"
    base = ct_norm[z]  # (H,W), uint8

    # 轉 BGR 方便畫彩色框
    base_bgr = cv2.cvtColor(base, cv2.COLOR_GRAY2BGR)
    ovl = base_bgr.copy()

    boxes = slice_nodules.get((case_key, z), [])
    for z_i, x1, y1, w, h, conf in boxes:
        p1 = (int(x1), int(y1))
        p2 = (int(x1 + w), int(y1 + h))
        cv2.rectangle(ovl, p1, p2, (0, 255, 0), thickness)
        cv2.putText(ovl, f"{conf:.2f}", (p1[0], max(0, p1[1] - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 1, cv2.LINE_AA)

    blended = cv2.addWeighted(base_bgr, 1.0, ovl, alpha, 0)

    if out_path:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        cv2.imwrite(out_path, blended)  # BGR 存檔

    if show:
        # 用 matplotlib 顯示，轉成 RGB
        plt.figure(figsize=(6,6))
        plt.imshow(cv2.cvtColor(blended, cv2.COLOR_BGR2RGB))
        plt.title(f"{case_key} | z={z} | n={len(boxes)}")
        plt.axis('off')
        plt.show()

In [24]:
# NMS+Cluster
def to_xyxy(b):  # b:[z,x1,y1,w,h,conf]
    z,x1,y1,w,h,conf = b
    return int(z),int(x1),int(y1),int(x1+w),int(y1+h),float(conf)

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2-ix1), max(0, iy2-iy1)
    inter = iw * ih
    area_a = max(0, ax2-ax1) * max(0, ay2-ay1)
    area_b = max(0, bx2-bx1) * max(0, by2-by1)
    union = area_a + area_b - inter + 1e-6
    return inter / union

def center_xyxy(a):
    x1,y1,x2,y2 = a
    return (0.5*(x1+x2), 0.5*(y1+y2))

# ========= 1) per-slice NMS =========
def per_slice_nms(slice_nodules):
    out = defaultdict(list)
    for (case_key, z), boxes in slice_nodules.items():
        # 轉 xyxy 並依 conf 排序（高→低）
        items = []
        for b in boxes:
            z_i,x1,y1,x2,y2,conf = to_xyxy(b)
            area = (x2-x1)*(y2-y1)
            if area < MIN_AREA_PX:   # 濾掉雜點
                continue
            items.append([x1,y1,x2,y2,conf])
        if not items:
            continue
        items.sort(key=lambda t: t[4], reverse=True)
        keep = []
        while items:
            x1,y1,x2,y2,conf = items.pop(0)
            keep.append([x1,y1,x2,y2,conf])
            items = [it for it in items if iou_xyxy((x1,y1,x2,y2), it[:4]) < IOU_THR_SLICE]
        # 可選：每片只留前 K
        if KEEP_TOPK_PER_SLICE is not None:
            keep = keep[:KEEP_TOPK_PER_SLICE]
        # 回寫為你的格式
        for x1,y1,x2,y2,conf in keep:
            out[(case_key, z)].append([int(z), int(x1), int(y1), int(x2-x1), int(y2-y1), float(conf)])
    return out

def link_across_slices(slice_nodules_dict):
    before_clusters = []
    clusters = []  # 每顆結節：{'case':..., 'zs':[], 'boxes':[], 'max_conf':..., 'rep':(z,x1,y1,w,h,conf)}
    # 先按 case 分組，再按 z 遞增
    by_case = defaultdict(list)
    for (case_key, z), boxes in slice_nodules_dict.items():
        for b in boxes:
            z_i,x1,y1,w,h,conf = b
            by_case[case_key].append((int(z_i), int(x1), int(y1), int(w), int(h), float(conf)))
    
    for case_key, items in by_case.items():
        # 按 z 再按 conf 排序
        items.sort(key=lambda t: (t[0], -t[5]))
        tracks = []  # 每個 track: {'last_z':, 'last_box':(x1,y1,x2,y2), 'zs':[], 'boxes':[], 'max_conf':}
        for z,x1,y1,w,h,conf in items:
            x2,y2 = x1+w, y1+h
            cx,cy = 0.5*(x1+x2), 0.5*(y1+y2)
            placed = False
            # 嘗試接到最近的一條軌
            best_i = -1; best_cost = 1e9
            for i,tr in enumerate(tracks):
                if abs(z - tr['last_z']) > MAX_Z_GAP:
                    continue
                tcx,tcy = center_xyxy(tr['last_box'])
                dz = abs(z - tr['last_z'])
                dxy = np.hypot(cx - tcx, cy - tcy)
                if dxy <= CENTER_THR_PX:
                    cost = dxy + 5*dz  # 簡單代價：平衡 xy 與 z
                    if cost < best_cost:
                        best_cost = cost; best_i = i
            if best_i >= 0:
                tr = tracks[best_i]
                tr['last_z'] = z
                tr['last_box'] = (x1,y1,x2,y2)
                tr['zs'].append(z)
                tr['boxes'].append((z,x1,y1,w,h,conf))
                tr['max_conf'] = max(tr['max_conf'], conf)
                placed = True
            if not placed:
                tracks.append(dict(
                    case=case_key,last_z=z, last_box=(x1,y1,x2,y2),
                    zs=[z], boxes=[(z,x1,y1,w,h,conf)], max_conf=conf
                ))
        # 濾掉太短的軌（只在 1 片出現的）
        for tr in tracks:
            before_clusters.append(dict(case=case_key, zs=tr['zs'], boxes=tr['boxes'],
                     max_conf=tr['max_conf'] ))
            if len(tr['zs']) < MIN_SLICE_LEN:
                continue
            # 代表框：取 conf 最大的那一片
            rep = max(tr['boxes'], key=lambda b: b[-1])
            clusters.append(dict(case=case_key, zs=tr['zs'], boxes=tr['boxes'],
                                 max_conf=tr['max_conf'], rep=rep))
    return before_clusters , clusters

In [25]:
slice_nodules_nms = per_slice_nms(slice_nodules)
before_clusters, clusters = link_across_slices(slice_nodules_nms)

print(f"原始片級偵測(去重前)片數：{len(slice_nodules)}")
print(f"Per-slice NMS 之後片數：{len(slice_nodules_nms)}")
print(f"跨片串連後結節顆數：{len(clusters)}")

# 你想只看每顆的代表框：
final_dets = defaultdict(list)  # key=case -> list of (z,x1,y1,w,h,conf)
for c in clusters:
    final_dets[c['case']].append(c['rep'])
for case, lst in final_dets.items():
    print(case, "->", len(lst), "nodules after 3D linking")


原始片級偵測(去重前)片數：1184
Per-slice NMS 之後片數：965
跨片串連後結節顆數：205
11161702 -> 9 nodules after 3D linking
15554940 -> 24 nodules after 3D linking
21658813 -> 9 nodules after 3D linking
21897647 -> 12 nodules after 3D linking
25053514 -> 12 nodules after 3D linking
29974122 -> 14 nodules after 3D linking
33455070 -> 12 nodules after 3D linking
37042242 -> 5 nodules after 3D linking
38433651 -> 6 nodules after 3D linking
39071719 -> 19 nodules after 3D linking
40198212 -> 5 nodules after 3D linking
42779819 -> 13 nodules after 3D linking
43204881 -> 15 nodules after 3D linking
58313873 -> 8 nodules after 3D linking
65141644 -> 23 nodules after 3D linking
86000068 -> 14 nodules after 3D linking
91295809 -> 5 nodules after 3D linking


In [26]:
# 每顆結節在哪些 slice；順便輸出代表片的 overlay
'''
for i, c in enumerate(clusters):
    print(f"[{i}] {c['case']} -> zs={sorted(c['zs'])}, len={len(c['zs'])}, max_conf={c['max_conf']:.3f}")
    z_rep, x1, y1, w, h, conf = c['rep']
    overlay_one_slice(c['case'], z_rep,
                      out_path=f"{OTHERS_PATH}/overlay_clusters/{c['case']}/nodule_{i}_z{z_rep:04d}.png",
                      show=False)
'''

'\nfor i, c in enumerate(clusters):\n    print(f"[{i}] {c[\'case\']} -> zs={sorted(c[\'zs\'])}, len={len(c[\'zs\'])}, max_conf={c[\'max_conf\']:.3f}")\n    z_rep, x1, y1, w, h, conf = c[\'rep\']\n    overlay_one_slice(c[\'case\'], z_rep,\n                      out_path=f"{OTHERS_PATH}/overlay_clusters/{c[\'case\']}/nodule_{i}_z{z_rep:04d}.png",\n                      show=False)\n'

In [27]:
before_clusters

[{'case': '11161702',
  'zs': [66],
  'boxes': [(66, 109, 253, 9, 9, 0.9993611574172974)],
  'max_conf': 0.9993611574172974},
 {'case': '11161702',
  'zs': [68, 69],
  'boxes': [(68, 113, 263, 6, 6, 0.9922879934310913),
   (69, 115, 264, 6, 7, 0.9975060224533081)],
  'max_conf': 0.9975060224533081},
 {'case': '11161702',
  'zs': [74],
  'boxes': [(74, 120, 282, 9, 12, 0.9984462857246399)],
  'max_conf': 0.9984462857246399},
 {'case': '11161702',
  'zs': [76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
  'boxes': [(76, 130, 278, 19, 20, 0.9991300106048584),
   (77, 131, 274, 26, 24, 0.9997289776802063),
   (78, 130, 274, 27, 25, 0.9997588992118835),
   (79, 131, 275, 26, 24, 0.9994947910308838),
   (80, 132, 276, 24, 23, 0.9998427033424377),
   (81, 132, 276, 25, 25, 0.9998040199279785),
   (82, 133, 275, 25, 25, 0.9992955923080444),
   (83, 134, 276, 24, 23, 0.9997098445892334),
   (84, 136, 277, 23, 23, 0.9997998476028442),
   (85, 136, 276, 24, 26, 0.9997220635414124),
   (86, 137, 

In [28]:
clusters

[{'case': '11161702',
  'zs': [68, 69],
  'boxes': [(68, 113, 263, 6, 6, 0.9922879934310913),
   (69, 115, 264, 6, 7, 0.9975060224533081)],
  'max_conf': 0.9975060224533081,
  'rep': (69, 115, 264, 6, 7, 0.9975060224533081)},
 {'case': '11161702',
  'zs': [76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
  'boxes': [(76, 130, 278, 19, 20, 0.9991300106048584),
   (77, 131, 274, 26, 24, 0.9997289776802063),
   (78, 130, 274, 27, 25, 0.9997588992118835),
   (79, 131, 275, 26, 24, 0.9994947910308838),
   (80, 132, 276, 24, 23, 0.9998427033424377),
   (81, 132, 276, 25, 25, 0.9998040199279785),
   (82, 133, 275, 25, 25, 0.9992955923080444),
   (83, 134, 276, 24, 23, 0.9997098445892334),
   (84, 136, 277, 23, 23, 0.9997998476028442),
   (85, 136, 276, 24, 26, 0.9997220635414124),
   (86, 137, 278, 14, 20, 0.9985390305519104),
   (87, 137, 289, 4, 7, 0.990950882434845)],
  'max_conf': 0.9998427033424377,
  'rep': (80, 132, 276, 24, 23, 0.9998427033424377)},
 {'case': '11161702',
  'zs': [89, 

In [29]:
# 每顆結節在哪些 slice；順便輸出代表片的 overlay
for i, c in enumerate(before_clusters):
    print(f"[{i}] {c['case']} -> zs={sorted(c['zs'])}, len={len(c['zs'])}, max_conf={c['max_conf']:.3f}")
    for k in range(len(c['boxes'])):
        z_rep, x1, y1, w, h, conf = c['boxes'][k]
        overlay_one_slice(c['case'], z_rep,
                          out_path=f"{OTHERS_PATH}/overlay_before_clusters/{c['case']}/nodule_{i}_z{z_rep:04d}.png",
                          show=False)

[0] 11161702 -> zs=[66], len=1, max_conf=0.999


GDCMSeriesFileNames (0x757fe40): /home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/data/11161702 is not a directory

GDCMSeriesFileNames (0x757fe40): No Series were found



RuntimeError: No DICOM series found in: /home/daniel77/LDCT/Unet_Densenet/notebook/1009_pipline/data/11161702

In [ ]:
!zip -r others/overlay_before_clusters.zip others/overlay_before_clusters/